## Задание 1

Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

In [1]:
import numpy as np

# код из Пример
def calc_logloss(y, y_pred):
    eps = 1e-15
    y_pred = np.clip(y_pred, eps, 1 - eps)
    err = -np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

## Задание 2

Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [4]:
import numpy as np
import matplotlib.pyplot as plt

# код из Пример
X = np.array([ [   1,    1,  50,    1],
               [   1,    1,  70,    1],
               [   1,    2,  75,    2],
               [   1,    5,  60,    1],
               [   1,    3, 145,    2],
               [   1,    0,  80,    1],
               [   1,    5, 150,    3],
               [   1,   10, 200,    3],
               [   1,    1,  45,    1],
               [   1,    2, 100,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

# код из Пример
def standard_scale(x):
    res = (x - x.mean()) / x.std()
    return res

# код из Пример
X_st = X.copy()
X_st[:, 2] = standard_scale(X[:, 2])

# код из Пример
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

# код из Пример, но ещё, кроме весов, возвращаем err, для того, чтобы за ним наблюдать
def eval_model(X, y, iterations, eta=1e-4):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]

    for i in range(iterations):
        z = np.dot(X, W)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)

        dQ = 1/n * X.T @ (y_pred - y)
        W -= eta * dQ
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W, err


In [9]:
def find_best_params(X, y, iteration_values, eta_values):
    best_loss = float('inf')
    best_params = {}

    for iterations in iteration_values:
        for eta in eta_values:
            weights, log_loss = eval_model(X, y, iterations, eta)

            if log_loss < best_loss:
                best_loss = log_loss
                best_params = {'iterations': iterations, 'eta': eta, 'weights': weights}

            print(f"Iterations: {iterations}, Eta: {eta}, Log Loss: {log_loss}")

    return best_params, best_loss

iteration_values = [50000]
eta_values = [1e-5,1e-4, 1e-3, 1e-2,5e-4]

best_params, best_loss = find_best_params(X_st, y, iteration_values, eta_values)

print(f"Best Log Loss: {best_loss}")
print(f"Best Parameters: {best_params}")

0 [ 0.49671036 -0.13827881  0.64768826  1.52302419] 1.1785958344356262
5000 [ 0.47821831 -0.20919509  0.64605912  1.49530997] 1.0557348309616592
10000 [ 0.46074075 -0.27648288  0.6439556   1.46891408] 0.9450161484874575
15000 [ 0.44435619 -0.33951752  0.64151879  1.44406788] 0.8476698479821569
20000 [ 0.42913512 -0.39751615  0.63899484  1.42106307] 0.7649859990682474
25000 [ 0.4151404  -0.44956012  0.63670925  1.40023268] 0.6980320275292
30000 [ 0.40240374 -0.49486993  0.63498186  1.38185393] 0.6468149936279607
35000 [ 0.39089227 -0.53319594  0.63401969  1.36602089] 0.6096598315659982
40000 [ 0.38050212 -0.56495364  0.63387127  1.35260312] 0.5836515866198949
45000 [ 0.37108704 -0.5910023   0.63446625  1.34131872] 0.5656992762209945
Iterations: 50000, Eta: 1e-05, Log Loss: 0.5532665943101582
0 [ 0.49667621 -0.13840939  0.6476858   1.52297324] 1.1785958344356262
5000 [ 0.36247418 -0.61238439  0.63568618  1.33181421] 0.5532531003505642
10000 [ 0.30085159 -0.70010366  0.66304526  1.2888086

## Задание 3

Создайте функцию, возвращающую предсказанную вероятность класса 1 (на вход подаются: W, который уже посчитан функцией eval_model и X; на выходе - массив с предсказанными вероятностями принадлежности объектов целевому классу).

In [6]:
def predict_proba(W, X):
    z = np.dot(X, W)
    y_pred_proba = sigmoid(z)
    return y_pred_proba

In [8]:
W, err = eval_model(X, y, iterations=50000, eta=0.001)
predicted_proba = predict_proba(W, X)
print(predicted_proba[:10])

0 [ 0.49621415 -0.1399643   0.60168854  1.52232986] 17.241019788925264
5000 [ 0.15538291 -0.43948832 -0.02589244  1.81467547] 0.6018531635355602
10000 [-0.17221211 -0.45454996 -0.025874    2.07579518] 0.550767228996118
15000 [-0.46625501 -0.46285861 -0.02548664  2.31123165] 0.5025811753011984
20000 [-0.73070329 -0.46716846 -0.02454112  2.52022805] 0.4596125052296237
25000 [-0.96855249 -0.47087075 -0.02258772  2.70510666] 0.4242222277297734
30000 [-1.18106975 -0.48492967 -0.02106222  2.87327827] 0.4056747966049932
35000 [-1.37072864 -0.50867234 -0.02131452  3.03414196] 0.3931848025355483
40000 [-1.54215894 -0.53195371 -0.02162815  3.1886792 ] 0.3824168880037796
45000 [-1.69833743 -0.55411332 -0.02200344  3.33723451] 0.37302253495025084
[0.48553058 0.37603541 0.90774573 0.07025106 0.53530026 0.46120394
 0.91365409 0.1627293  0.5135555  0.84887058]


## Задание 4

Создайте функцию, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X; на выходе - массив классов, т.е. 0 или 1 для каждого объекта).

In [10]:
def predict(w, X):

    m = X.shape[0]

    y_predicted = np.zeros(m)

    A = np.squeeze(sigmoid(np.dot(X, w)))

    for i in range(A.shape[0]):
        if (A[i] > 0.5):
            y_predicted[i] = 1
        elif (A[i] <= 0.5):
            y_predicted[i] = 0

    return y_predicted

In [13]:
W, err = eval_model(X, y, iterations=50000, eta=0.001)
predicted_proba = predict_proba(W, X)
predicted = predict(W,X);
W
# Пример вывода предсказанных вероятностей для первых 10 объектов
print(f"Predicted result: {predicted}")
print(f"Real: {y}")
print(f"Probability of first class: {predicted_proba}")

0 [ 0.49621415 -0.1399643   0.60168854  1.52232986] 17.241019788925264
5000 [ 0.15538291 -0.43948832 -0.02589244  1.81467547] 0.6018531635355602
10000 [-0.17221211 -0.45454996 -0.025874    2.07579518] 0.550767228996118
15000 [-0.46625501 -0.46285861 -0.02548664  2.31123165] 0.5025811753011984
20000 [-0.73070329 -0.46716846 -0.02454112  2.52022805] 0.4596125052296237
25000 [-0.96855249 -0.47087075 -0.02258772  2.70510666] 0.4242222277297734
30000 [-1.18106975 -0.48492967 -0.02106222  2.87327827] 0.4056747966049932
35000 [-1.37072864 -0.50867234 -0.02131452  3.03414196] 0.3931848025355483
40000 [-1.54215894 -0.53195371 -0.02162815  3.1886792 ] 0.3824168880037796
45000 [-1.69833743 -0.55411332 -0.02200344  3.33723451] 0.37302253495025084
Predicted result: [0. 0. 1. 0. 1. 0. 1. 0. 1. 1.]
Real: [0. 0. 1. 0. 1. 0. 1. 0. 1. 1.]
Probability of first class: [0.48553058 0.37603541 0.90774573 0.07025106 0.53530026 0.46120394
 0.91365409 0.1627293  0.5135555  0.84887058]


## Задание 5

*Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score

In [17]:
# доля правильно классифицированных объектов от общего числа объектов.
def accuracy(y_true, y_pred):
    return np.mean(y_true == y_pred)

def confusion_matrix(y_true, y_pred):
    TP = np.sum((y_true == 1) & (y_pred == 1))  # верно как 1
    TN = np.sum((y_true == 0) & (y_pred == 0))  # верно как 0
    FP = np.sum((y_true == 0) & (y_pred == 1))  # неверно как 1
    FN = np.sum((y_true == 1) & (y_pred == 0))  # неверно как 0

    return np.array([[TN, FP], [FN, TP]])

# доля правильно классифицированных 1 от общего числа предсказанных 1.
def precision(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    TP = cm[1, 1]
    FP = cm[0, 1]

    return TP / (TP + FP) if (TP + FP) > 0 else 0

# доля правильно классифицированных 1 от общего числа реальных 1.
def recall(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    TP = cm[1, 1]
    FN = cm[1, 0]

    return TP / (TP + FN) if (TP + FN) > 0 else 0

#гармоническое среднее (обр. ср.) между точностью и полнотой
def f1_score(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)

    return 2 * (prec * rec) / (prec + rec) if (prec + rec) > 0 else 0

In [18]:
W, err = eval_model(X_st, y, iterations=50000, eta=0.001)
predicted_proba = predict_proba(W, X_st)
predicted = predict(W, X_st)

acc = accuracy(y, predicted)
cm = confusion_matrix(y, predicted)
prec = precision(y, predicted)
rec = recall(y, predicted)
f1 = f1_score(y, predicted)

print(f"Accuracy: {acc}")
print(f"Confusion Matrix:\n{cm}")
print(f"Precision: {prec}")
print(f"Recall: {rec}")
print(f"F1 Score: {f1}")

0 [ 0.49633477 -0.13971518  0.64766116  1.52246371] 1.1785958344356262
5000 [ 0.0403965  -0.72055655  0.85796361  1.32692568] 0.49100580319420767
10000 [-0.1669251  -0.75343521  0.95139369  1.46315236] 0.4764710362288375
15000 [-0.33555542 -0.78990057  0.98208702  1.60956057] 0.4660116896153913
20000 [-0.49166777 -0.82093541  0.98622379  1.74689957] 0.4571592093330765
25000 [-0.64188403 -0.84590689  0.97563061  1.87296157] 0.4493155255168114
30000 [-0.78798636 -0.86575743  0.95535381  1.98903671] 0.442187479414062
35000 [-0.93053319 -0.88153047  0.92819883  2.09680376] 0.4356015801308559
40000 [-1.06975797 -0.89409829  0.89602171  2.19771312] 0.4294483493013447
45000 [-1.20580028 -0.90414943  0.86016469  2.29292551] 0.4236555083479012
Accuracy: 0.8
Confusion Matrix:
[[4 1]
 [1 4]]
Precision: 0.8
Recall: 0.8
F1 Score: 0.8000000000000002
